In [41]:
import os
import subprocess
import torch
from torchvision import datasets, transforms
import torchvision.transforms.v2
from PIL import Image
import ipynbname
import json
import shutil

In [36]:
path_home = "/Users/hadare/Documents/CodingProjects/SyntheticEvaluation"
path_img =  path_home + "/images"
path_dataset = path_home + "/dataset"
path_exp = path_home + "/experiments"
raw_data = path_home + "/data/mnist_images"

In [25]:
# Create directories to save MNIST images
mnist_dir = raw_data
os.makedirs(mnist_dir, exist_ok=True)

# Download MNIST dataset
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    # transforms.Grayscale(num_output_channels=1),  # In case the images are not grayscale
    transforms.v2.RGB(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
mnist_dataset = datasets.MNIST(root='data', train=True, transform=transform, download=True)

# Save each image as a separate file
for idx, (img, label) in enumerate(mnist_dataset):
    img_path = os.path.join(mnist_dir, f'{label}_{idx}.png')
    img = transforms.ToPILImage()(img)
    img.save(img_path)


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:04<00:00, 2465210.66it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 187442.85it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:04<00:00, 349873.46it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 1126184.01it/s]


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [26]:

# Test all files in path_img for size and format:
from os import listdir
from os.path import isfile, join
import os
from PIL import Image   
from tqdm.notebook import tqdm


IMAGE_PATH = raw_data
files = [f for f in listdir(IMAGE_PATH) if isfile(join(IMAGE_PATH, f))]

base_size = None
consistent_files = 0
inconsistent_size_files = 0
inconsistent_color_files = 0
total_files = len(files)

# Set the interval for status updates (e.g., every 100 files)
update_interval = int(total_files/5)

# Create a progress bar
progress_bar = tqdm(total=total_files, desc="Processing files", unit="file")

for i, file in enumerate(files, 1):
    file_path = os.path.join(IMAGE_PATH, file)
    img = Image.open(file_path)
    sz = img.size
    
    if base_size is None:
        base_size = sz
    
    if sz != base_size:
        inconsistent_size_files += 1
    elif img.mode != 'RGB':
        inconsistent_color_files += 1
    else:
        consistent_files += 1
    
    # Update the progress bar
    progress_bar.update(1)
    
    # Print status update every x files
    if i % update_interval == 0:
        print(f"Processed {i}/{total_files} files...")

# Close the progress bar
progress_bar.close()

# Print completion stats
print("\nProcessing complete. Summary:")
print(f"Total files processed: {total_files}")
print(f"Consistent files: {consistent_files}")
print(f"Files with inconsistent size: {inconsistent_size_files}")
print(f"Files with inconsistent color format: {inconsistent_color_files}")
print(f"Base size: {base_size}")



Processing files:   0%|          | 0/60000 [00:00<?, ?file/s]

Processed 12000/60000 files...
Processed 24000/60000 files...
Processed 36000/60000 files...
Processed 48000/60000 files...
Processed 60000/60000 files...

Processing complete. Summary:
Total files processed: 60000
Consistent files: 60000
Files with inconsistent size: 0
Files with inconsistent color format: 0
Base size: (32, 32)


In [42]:
# divide into subdirectories

def distribute_files(src_dir):
    # Create class directories if they don't exist
    for class_num in range(10):
        class_dir = os.path.join(src_dir, f"class_{class_num}")
        if not os.path.exists(class_dir):
            os.makedirs(class_dir)

    # Distribute files into their respective class directories
    for filename in os.listdir(src_dir):
        src_path = os.path.join(src_dir, filename)
        if os.path.isfile(src_path):
            try:
                file_class = filename.split("_")[0][0]
                if file_class.isdigit():
                    dst_path = os.path.join(src_dir, f"class_{file_class}", filename)
                    shutil.move(src_path, dst_path)
            except (IndexError, ValueError):
                # Handle files that don't follow the expected naming convention
                print(f"Skipping file: {filename} (doesn't follow the expected naming convention)")

source_directory = raw_data
distribute_files(source_directory)

In [ ]:
# Format for the dataset.json file:

# {
#     "labels":
#         [
#             ["folder1/1.jpg", 0], ["folder1/2.jpg", 0], ["folder1/3.jpg", 0], 
#             ["folder2/4.jpg", 1], ["folder2/5.jpg", 1], ["folder2/6.jpg", 1], 
#             ["folder3/7.jpg", 2], ["folder3/8.jpg", 2], ["folder3/9.jpg", 2], 
#         ]
# }

In [43]:
def generate_labels_json(base_dir,output_dir, output_file_name):
    labels_data = {"labels": []}
    
    # Walk through the base directory and its subdirectories
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            # Only include image files (you can adjust this based on file extensions)
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff')):
                # Extract the class label from the first digit of the file name
                try:
                    label = int(file[0])  # First digit of the file name
                except ValueError:
                    # Skip files that do not start with a digit
                    continue
                
                # Create relative file path from the base directory
                file_path = os.path.relpath(os.path.join(root, file), base_dir)
                
                # Add the file path and corresponding label to the list
                labels_data["labels"].append([file_path, label])


    os.makedirs(output_dir, exist_ok=True)
    output_file_path = os.path.join(output_dir, output_file_name)

    # Write the labels to the output JSON file
    with open(output_file_path, 'w') as f:
        json.dump(labels_data, f, indent=4)

base_directory = raw_data
print(base_directory)
generate_labels_json(base_directory, base_directory, "dataset.json")


/Users/hadare/Documents/CodingProjects/SyntheticEvaluation/data/mnist_images


In [44]:
import json
import pandas as pd
from collections import Counter
import random

def load_data(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
    return data['labels']

def create_dataframe(data):
    df = pd.DataFrame(data, columns=['file_path', 'label'])
    return df

def subset_data(df, subset_size, seed=None):
    if seed:
        random.seed(seed)

    class_counts = Counter(df['label'])
    num_classes = len(class_counts)
    class_subset_sizes = {class_id: int(subset_size / num_classes) for class_id in class_counts}

    subsets = []
    for class_id, count in class_counts.items():
        class_df = df[df['label'] == class_id]
        class_subset = class_df.sample(n=class_subset_sizes[class_id], replace=False, random_state=seed)
        subsets.append(class_subset)

    subset_df = pd.concat(subsets, ignore_index=True)
    return subset_df

def save_data(subset_df, output_file):
    data = {
        "labels": subset_df.values.tolist()
    }
    with open(output_file, 'w') as f:
        json.dump(data, f, indent=4)

# Example usage
input_file = raw_data + "/dataset.json"
output_file = raw_data + "/dataset_subset.json"
subset_size = 1000
seed_value = 42

data = load_data(input_file)
df = create_dataframe(data)
subset_df = subset_data(df, subset_size, seed=seed_value)
save_data(subset_df, output_file)

In [50]:
import json
import pandas as pd
from collections import Counter
import random
from tqdm import tqdm

def load_data(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
    return data['labels']

def create_dataframe(data):
    df = pd.DataFrame(data, columns=['file_path', 'label'])
    return df

def split_data(df, train_ratio, seed=None):
    if seed:
        random.seed(seed)

    # Split the data into train and test sets
    grouped = df.groupby('label')
    train_data = []
    test_data = []

    for _, group in tqdm(grouped, desc='Splitting data', unit='class'):
        group = group.sample(frac=1, random_state=seed).reset_index(drop=True)  # Shuffle the group
        train_size = int(len(group) * train_ratio)
        train_data.extend(group.iloc[:train_size].to_dict('records'))
        test_data.extend(group.iloc[train_size:].to_dict('records'))

    train_df = pd.DataFrame(train_data, columns=['file_path', 'label'])
    test_df = pd.DataFrame(test_data, columns=['file_path', 'label'])

    return train_df, test_df

def save_data(data_df, output_file):
    data = {
        "labels": data_df.values.tolist()
    }
    with open(output_file, 'w') as f:
        json.dump(data, f, indent=4)

# Example usage
input_file = raw_data + "/dataset.json"
# output_file = raw_data + "/dataset_subset.json"
output_dir = raw_data

# input_file = 'path/to/input.json'
# output_dir = 'path/to/output_dir'
train_ratio = 0.8  # 80% for train, 20% for test
seed_value = 42

print(f"Loading data from {input_file}")
data = load_data(input_file)
df = create_dataframe(data)

print(f"Splitting data into train and test sets with train ratio {train_ratio} and seed {seed_value}")
train_df, test_df = split_data(df, train_ratio, seed=seed_value)

train_output_file = f"{output_dir}/train_data.json"
test_output_file = f"{output_dir}/test_data.json"

print(f"Saving train data to {train_output_file}")
save_data(train_df, train_output_file)

print(f"Saving test data to {test_output_file}")
save_data(test_df, test_output_file)

print("Train data statistics:")
print(train_df['label'].value_counts().sort_index())

print("Test data statistics:")
print(test_df['label'].value_counts().sort_index())

Loading data from /Users/hadare/Documents/CodingProjects/SyntheticEvaluation/data/mnist_images/dataset.json
Splitting data into train and test sets with train ratio 0.8 and seed 42


Splitting data: 100%|██████████| 10/10 [00:00<00:00, 255.96class/s]

Saving train data to /Users/hadare/Documents/CodingProjects/SyntheticEvaluation/data/mnist_images/train_data.json
Saving test data to /Users/hadare/Documents/CodingProjects/SyntheticEvaluation/data/mnist_images/test_data.json
Train data statistics:
label
0    4738
1    5393
2    4766
3    4904
4    4673
5    4336
6    4734
7    5012
8    4680
9    4759
Name: count, dtype: int64
Test data statistics:
label
0    1185
1    1349
2    1192
3    1227
4    1169
5    1085
6    1184
7    1253
8    1171
9    1190
Name: count, dtype: int64


In [49]:
import json
import pandas as pd
from collections import Counter
import random
from tqdm import tqdm

def load_data(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
    return data['labels']

def create_dataframe(data):
    df = pd.DataFrame(data, columns=['file_path', 'label'])
    return df

def subset_data(df, subset_size, seed=None):
    if seed:
        random.seed(seed)

    class_counts = Counter(df['label'])
    num_classes = len(class_counts)
    class_subset_sizes = {class_id: int(subset_size / num_classes) for class_id in class_counts}

    subsets = []
    for class_id, count in tqdm(class_counts.items(), desc='Subsetting classes', unit='class'):
        class_df = df[df['label'] == class_id]
        class_subset = class_df.sample(n=class_subset_sizes[class_id], replace=False, random_state=seed)
        subsets.append(class_subset)

    subset_df = pd.concat(subsets, ignore_index=True)
    return subset_df

def save_data(subset_df, output_file):
    data = {
        "labels": subset_df.values.tolist()
    }
    with open(output_file, 'w') as f:
        json.dump(data, f, indent=4)

# Example usage
subset_size = 4000
seed_value = 42
input_file = raw_data + "/dataset.json"
# output_file = raw_data + "/dataset_subset.json"
output_file = raw_data + f"/dataset_subset_size{subset_size}_seed{seed_value}.json"

print(f"Loading data from {input_file}")
data = load_data(input_file)
df = create_dataframe(data)

print(f"Subsetting data with size {subset_size} and seed {seed_value}")
subset_df = subset_data(df, subset_size, seed=seed_value)

print(f"Saving data to {output_file}")
save_data(subset_df, output_file)

print("Subset statistics:")
print(subset_df['label'].value_counts().sort_index())

Loading data from /Users/hadare/Documents/CodingProjects/SyntheticEvaluation/data/mnist_images/dataset.json
Subsetting data with size 4000 and seed 42


Subsetting classes: 100%|██████████| 10/10 [00:00<00:00, 1630.00class/s]

Saving data to /Users/hadare/Documents/CodingProjects/SyntheticEvaluation/data/mnist_images/dataset_subset_size4000_seed42.json
Subset statistics:
label
0    400
1    400
2    400
3    400
4    400
5    400
6    400
7    400
8    400
9    400
Name: count, dtype: int64
